In [29]:
from tqdm import tqdm 
from colorama import Style,Fore
import os
from dotenv import load_dotenv
from datetime import datetime,timezone
load_dotenv()

True

In [30]:
from googleapiclient.discovery import build

youtube = build("youtube", "v3", developerKey=os.getenv("YOUTUBE_API_KEY1"))

In [31]:
def getComments(video_id):
    comments_dict = {}
    request = youtube.commentThreads().list(
        part='snippet,replies',
        videoId=video_id,
        maxResults=100,
        textFormat='plainText'
    )

    while request:
        response = request.execute()
        for item in response.get('items', []):
            top_comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            replies = []

            if 'replies' in item:
                for reply in item['replies']['comments']:
                    reply_text = reply['snippet']['textDisplay']
                    replies.append(reply_text)

            comments_dict[top_comment] = replies

        request = youtube.commentThreads().list_next(request, response)

    return comments_dict

def calculateComments(comments_dict):
    total = 0
    for top_comment, replies in comments_dict.items():
        total += 1
        total += len(replies)
    return total

In [32]:
comments = getComments("yTQYw5Omdso")

In [33]:
calculateComments(comments)

14

In [36]:
def getVideoMetrics(video_id):
    video_response = youtube.videos().list(
        part='statistics,snippet',
        id=video_id
    ).execute()

    if not video_response['items']:
        return None

    video = video_response['items'][0]
    stats = video['statistics']

    views = int(stats.get('viewCount', 0))
    likes = int(stats.get('likeCount', 0))

    comments_dict = getComments(video_id)

    metrics = {
        'id_video': video_id,
        'date_releve': datetime.now(timezone.utc).date().isoformat(),
        'nombre_vues': views,
        'nombre_likes': likes,
        'commentaires': comments_dict
    }

    return metrics

In [37]:
getVideoMetrics("yTQYw5Omdso")

{'id_video': 'yTQYw5Omdso',
 'date_releve': '2025-05-06',
 'nombre_vues': 6289,
 'nombre_likes': 287,
 'commentaires': {'La nature est  belle  ...chaque jour elle nous émerveille, tout au long des saisons, elle me touche. Très jolie vidéo, merci.  Vos vidéos au jardin me  manquent. A toutes mes connaissances en jardin, je donne votre jardin en exemple...': [],
  'merci pour cette vidéo 👍👍👍\r\nbonne continuation a bientôt bisou 😘😘\r\nChico😘': [],
  'Tous les sens en «\xa0réveil\xa0», …\nMerci pour cette parenthèse \n👀 🥬  🌺  🎶 👅': [],
  'Vraiment magnifique bravo 😉👍': [],
  'Votre jardin est une source d’inspiration.': [],
  'C’est super beau ! Merci !': [],
  'Chaque vidéo est un vrai petit bijou avec le potager comme écrin et Marie en orfèvre.': [],
  "Ici au Québec, c'est plutôt du -30 degrés. Alors il faut patienter pour cultiver notre potager. Merci pour vos beaux livres inspirants.": [],
  'Tout en délicatesse 🙏💕': [],
  'Toujours aussi magnifique. Merci pour ce moment de plaisir.'